# Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
library(brms)
library(gridExtra)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
source(paste0(helpers_path, '01_clean_behavioral_data.R'))

rm(data_bc_clean)

data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

# Hierarchy over days

Previous version of this nb had all these analyses using estimates with a hierarchy over subjects. Since this makes less sense for this experiment I changed it all to use estimates with a hierarchy over days instead.

Read in parameter estimates. This contains all 30000 samples per parameter for each subject and stim type across days. Takes a while to read in.

In [ ]:
fn = file.path(here(), "inputs",'yn_sub_hddm_mcmc_draws.csv')

if(file.exists(fn)){
  all_results_df = read.csv(fn)
  
} else{
  stim_types = c("HT", "RE")
  subnums = c(601, 609, 611, 619, 621, 629)
  
  in_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/jags_out'
  
  all_results_df = data.frame()
  
  for(i in 1:length(stim_types)) {
    for(j in 1:length(subnums)) {
      
      cur_stim_type = stim_types[i]
      cur_sub = subnums[j]
      
      # fn = file.path(in_path, paste0('results_YN_HDDM_FIT_', cur_stim_type,'_day_', cur_day,'.RData'))
      fn = file.path(in_path, paste0('results_YN_HDDM_FIT_', cur_stim_type,'_sub-', cur_sub,'.RData'))
      
      if(file.exists(fn)) {
        
        load(fn)
        
        cur_results_df = data.frame()
        for(k in 1:length(results$mcmc)){
          cur_chain = as.data.frame(results$mcmc[[k]])
          cur_results_df = rbind(cur_results_df, cur_chain)
        }
        
        cur_results_df = cur_results_df %>%
          gather(par, estimate) %>%
          filter(grepl("\\[", par)) %>%
          mutate(type = cur_stim_type,
                 subnum = cur_sub)
        
        cur_results_df = cur_results_df %>%
          separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
          separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
          select(-a) %>%
          mutate(d = paste0('p[',d)) %>%
          mutate(b = ifelse(is.na(b), d, b)) %>%
          select(-d, -par) %>%
          rename(par_name = c, par_group = b) %>%
          mutate(par_group = gsub("p\\[", "", par_group),
                 par_group = gsub("\\]", "", par_group)) %>%
          filter(par_name != "deviance") %>%
          filter(par_group != "kappa" & par_group != "pr") %>%
          filter(par_group != "mu") %>% #Not showing group estimates in this plot
          rename(day = par_group) %>%
          mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name)))) %>%
          mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt")),
                 day = as.numeric(day))
        
        all_results_df = rbind(all_results_df, cur_results_df)
        
      } else {
        next
      }
    }
  }
  fn = file.path(here(), "inputs",'yn_sub_hddm_mcmc_draws.csv')
  write.csv(all_results_df, fn, row.names = F)
  rm(cur_chain, cur_results_df, results, cur_sub, cur_stim_type, subnums, fn, i, j, k, stim_types)
}




## Correlations between estimates

From all parameters from all chains extract only the drift rates and sigmas

In [ ]:
all_d_sigma = all_results_df %>%
  filter(par_name %in% c("d", "sigma"))

rm(all_results_df)

Compute correlation between the 30000 d and sigmas for each subject, day and stim type

In [ ]:
subs = c('601', '609', '611', '619', '621', '629')
types = c('HT', 'RE')
days = seq(1,11,1)

d_sigma_cors =  data.frame(subnum = NA, type = NA, day = NA, cor = NA)

for (i in 1:length(subs)){
  for (j in 1:length(types)){
    for (k in 1:length(days)){
      
      cur_sub = subs[i]
      cur_type = types[j]
      cur_day = days[k]
      
      cur_ds = all_d_sigma %>%
        filter(subnum == cur_sub & type == cur_type & day == cur_day & par_name == "d") %>%
        select(estimate)
      
      cur_sigmas = all_d_sigma %>%
        filter(subnum == cur_sub & type == cur_type & day == cur_day & par_name == "sigma") %>%
        select(estimate)
      
      cur_cor = round(cor(cur_ds, cur_sigmas), 3)
      
      cur_row = data.frame(subnum = cur_sub, type = cur_type, day = cur_day, cor = as.numeric(cur_cor))
      
      d_sigma_cors = rbind(d_sigma_cors, cur_row)
    }
  }
}


d_sigma_cors = d_sigma_cors %>%
  drop_na()

In [ ]:
p = d_sigma_cors %>%
  ggplot(aes(type, cor, fill=type))+
  geom_boxplot()+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  scale_fill_brewer(palette = "Dark2")+
  labs(x = "", fill = "", y = "Correlation between d and sigma estimates")

# ggsave(file=paste0(fig_out_path, 'yn_ddm_sub_d_sigma_cor.jpg'), p, height = 5, width = 5, units="in")
p

## Posterior predictive checks

### Predicted data accounting for posterior uncertainty

With 1000 samples from posteriors to account for uncertainty in estimates.

In [ ]:

# Takes a while
inpath = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/sim_out'

# conds = c('HT')
# subs = c('609','621')

conds = c('HT', 'RE')
subs = c('601', '609','611', '619', '621', '629')
days = seq(1, 11, 1)


for (k in 1:length(conds)){
  cur_cond = conds[k]
  
  for (j in 1:length(subs)){
    cur_sub = subs[j]
    
    for (i in 1:length(days)){
      cur_day = days[i]
      
      fn = file.path(inpath, paste0("yn_sub_hddm_sim_sub-", cur_sub, "_", cur_cond, "_day-", cur_day, ".csv"))
      
      if(!file.exists(fn)){
        next
      } else{
        tmp = read.csv(fn)
      }
      
      
      if(i == 1){
        sub_cond_sim_data = tmp
      } else {
        sub_cond_sim_data = rbind(sub_cond_sim_data, tmp)
      }
    }
    fill_col = ifelse(cur_cond == "HT", "#1B9E77", "#D95F02")
    
    p = sub_cond_sim_data %>%
      filter(rt < 3 & pred_rt < 3) %>%
      mutate(rt = ifelse(yesChosen == 1, rt, rt*(-1)),
             pred_rt = ifelse(pred_yesChosen == 1, pred_rt, pred_rt*(-1)), 
             day = paste0("Day ", day),
             day = factor(day, levels = paste0("Day ", seq(1,11, 1)))) %>%
      ggplot()+
      geom_density(aes(x = pred_rt), position = "identity", alpha = .5) +
      geom_histogram(aes(x = rt, y = ..density..), bins = 30,  position = "identity", alpha = .5, color = NA, fill = fill_col)+
      facet_grid(subnum ~ day)+
      theme(panel.grid = element_blank(),
            axis.text = element_blank(),
            axis.ticks = element_blank())+
      geom_vline(aes(xintercept = 0), color = "gray") +
      labs(y = "", x = "")
    
    fn = paste0(fig_out_path,'yn_sub_hddm_sim_rt_ppc_sub-',cur_sub, '_cond-',cur_cond, '.jpg')
    print(paste("Saving", fn))
    ggsave(file = fn, p, height = 2, width = 18, units="in")
    
  }
}



### Mean a posteriori data

Does mean a posteriori data capture other choice and RT patterns?

Read in parameter posterior summaries and organize them

In [ ]:
stim_types = c("HT", "RE")
subnums = c(601, 609, 611, 619, 621, 629)

in_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/jags_out'

yn_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(subnums)){
    
    cur_stim_type = stim_types[i]
    cur_sub = subnums[j]
    # summary_YN_HDDM_FIT_RE_sub-611.csv
    
    fn = file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_sub-', cur_sub,'.csv'))
    
    if(file.exists(fn)){
      cur_summary = read.csv(fn)
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               subnum = cur_sub)
      
      yn_ddm_summary = rbind(yn_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}

In [ ]:
yn_ddm_summary = yn_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, subnum, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_group = gsub("p\\[", "", par_group),
         par_group = gsub("\\]", "", par_group)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  rename(day = par_group) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name)))) %>%
  mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt")),
         day = as.numeric(day))

Simulation helper functions

In [ ]:
sim_trial = function(trial_vdiff, sampled_sigma, sampled_d, sampled_ndt, sampled_bias){
  
  # Multiplying boundary separation and drift rate my sigma to account for that parameter
  trial_drift = sampled_sigma * (sampled_d * trial_vdiff)
  
  if(abs(trial_drift) < 10){
    cur_pred = RWiener::rwiener(1, 2*sampled_sigma, sampled_ndt, sampled_bias, trial_drift)
  } else {
    print("d too large. Sampling RT.")
    pq = sampled_ndt + abs(rnorm(1, mean = 0, sd = 0.01))
    pr = ifelse(trial_drift>10, factor("upper", levels = c("upper", "lower")), factor("lower", levels = c("upper", "lower")))
    cur_pred = data.frame(q = pq, resp = pr)
  }
  return(cur_pred)
  
}

sim_subj_cond_day_once = function(cur_stims, sampled_sigma, sampled_d, sampled_ndt, sampled_bias){
  
  pred_data = cur_stims
  
  normMax = 1
  normMin = -1
  # Already filtered for sub, day and stim type so don't need to group to make sure vdiff ranges between [-1,1]
  pred_data = pred_data %>%
    mutate(rawVDiff = possiblePayoff - reference,
           normVDiff =  (normMax - normMin) / (max(rawVDiff) - min(rawVDiff)) * (rawVDiff - max(rawVDiff)) + (normMax) )
  
  pred_data$sampled_sigma = sampled_sigma
  pred_data$sampled_d = sampled_d
  pred_data$sampled_ndt = sampled_ndt
  pred_data$sampled_bias = sampled_bias
  pred_data$pred_rt = NA
  pred_data$pred_yesChosen = NA
  
  for(i in 1:nrow(cur_stims)){
    
    trial_vdiff = pred_data$normVDiff[i]
    cur_pred = sim_trial(trial_vdiff, sampled_sigma, sampled_d, sampled_ndt, sampled_bias)
    
    pred_data$pred_rt[i] = cur_pred$q
    pred_data$pred_yesChosen[i] = cur_pred$resp
    
  }
  
  return(pred_data)
}

In [ ]:
conds = c('HT', 'RE')
subs = c('601', '609','611', '619', '621', '629')
days = seq(1, 11, 1)

for(i in 1:length(conds)){
  cur_type = conds[i]
  
  for (j in 1:length(subs)){
    cur_sub = subs[j]
    
    for (k in 1:length(days)){
      cur_day = days[k]
      
      cur_stims = data_yn_clean %>%
        filter(subnum == cur_sub & day == cur_day & type == cur_type) %>%
        select(subnum, day, type, possiblePayoff, reference, yesChosen, rt)
      
      cur_pars = yn_ddm_summary %>%
        filter(subnum == cur_sub & day == cur_day & stim_type == cur_type) %>%
        select(par_name, Mean) %>%
        spread(par_name, Mean)
      
      cur_pred = sim_subj_cond_day_once(cur_stims, cur_pars$sigma, cur_pars$d, cur_pars$ndt, cur_pars$bias)
      
      if(!exists("sim_yn_map")){
        sim_yn_map = cur_pred
      } else{
        sim_yn_map = rbind(sim_yn_map, cur_pred)
      }
    }
  }
}

In [ ]:
rm(cur_pars, cur_pred, cur_stims, cur_summary, cur_day, cur_stim_type, cur_sub, cur_type)
rm(i, j, k)

Add columns for the upcoming analyses

In [ ]:
sim_yn_map = sim_yn_map %>%
  mutate(pred_yesChosen = ifelse(pred_yesChosen == 1, pred_yesChosen, 0),
         val_diff = possiblePayoff - reference,
         day_std = (day - mean(day))/sd(day)) %>%
  group_by(subnum) %>%
  mutate(val_diff_std = (val_diff - mean(val_diff))/sd(val_diff),
         abs_val_diff = abs(val_diff),
         abs_val_diff_std = (abs_val_diff - mean(abs_val_diff))/sd(abs_val_diff))

sim_yn_map = sim_yn_map %>%
  mutate(val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")),
         abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

data_yn_clean = data_yn_clean %>%
  mutate(abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")),
         abs_val_diff_bin_midpoint = round(abs(val_diff)/50)*50)

Check RT/Choice plots look similar to the simulated data using samples from the posterior

In [ ]:
cur_cond = 'HT'
fill_col = ifelse(cur_cond == "HT", "#1B9E77", "#D95F02")

sim_yn_map %>%
  filter(type == cur_cond) %>%
  filter(rt < 3 & pred_rt < 3) %>%
  mutate(rt = ifelse(yesChosen == 1, rt, rt*(-1)),
         pred_rt = ifelse(pred_yesChosen == 1, pred_rt, pred_rt*(-1)), 
         day = paste0("Day ", day),
         day = factor(day, levels = paste0("Day ", seq(1,11, 1)))) %>%
  ggplot()+
  geom_density(aes(x = pred_rt), position = "identity", alpha = .5) +
  geom_histogram(aes(x = rt, y = ..density..), bins = 30,  position = "identity", alpha = .5, color = NA, fill = fill_col)+
  facet_grid(subnum ~ day)+
  theme(panel.grid = element_blank(),
        axis.text = element_blank(),
        axis.ticks = element_blank())+
  geom_vline(aes(xintercept = 0), color = "gray") +
  labs(y = "", x = "")

In [ ]:
cur_cond = 'RE'
fill_col = ifelse(cur_cond == "HT", "#1B9E77", "#D95F02")

sim_yn_map %>%
  filter(type == cur_cond) %>%
  filter(rt < 3 & pred_rt < 3) %>%
  mutate(rt = ifelse(yesChosen == 1, rt, rt*(-1)),
         pred_rt = ifelse(pred_yesChosen == 1, pred_rt, pred_rt*(-1)), 
         day = paste0("Day ", day),
         day = factor(day, levels = paste0("Day ", seq(1,11, 1)))) %>%
  ggplot()+
  geom_density(aes(x = pred_rt), position = "identity", alpha = .5) +
  geom_histogram(aes(x = rt, y = ..density..), bins = 30,  position = "identity", alpha = .5, color = NA, fill = fill_col)+
  facet_grid(subnum ~ day)+
  theme(panel.grid = element_blank(),
        axis.text = element_blank(),
        axis.ticks = element_blank())+
  geom_vline(aes(xintercept = 0), color = "gray") +
  labs(y = "", x = "")

What are the other choice and RT patterns the predicted data should capture?

### Value difference on choice

In [ ]:
sim_group_summary = sim_yn_map %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type) %>%
  summarise(mean_y = mean(pred_yesChosen),
            .groups="keep") %>%
  mutate(data = "Sim")

true_group_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, val_diff_bin_str, type) %>%
  summarise(mean_y = mean(yesChosen),
            .groups="keep") %>%
  mutate(data = "True")

In [ ]:
p = ggplot() +
  geom_hline(data = true_group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = true_group_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = true_group_summary, aes(val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = true_group_summary, aes(val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task (True)", alpha="", size="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))


# ggsave(file=paste0(fig_out_path, 'ynSimCompScale_group_pYesOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
  geom_hline(data = sim_group_summary, aes(yintercept =.5), color="gray")+
  geom_vline(data = sim_group_summary, aes(xintercept = 5), color="gray")+
  geom_point(data = sim_group_summary, aes(val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = sim_group_summary, aes(val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task (Sim)", alpha="", size="")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))


# ggsave(file=paste0(fig_out_path, 'simsubyn_group_pYesOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

Choice:
- Slope of value difference on choice logit always positive?
- Slope of value difference on choice logit smaller for RE than HT?
- Slope of value difference on choice logit increases with days?
- Slope of value difference on choice logit increases less for RE than HT with days?


In [ ]:
sim_yn_map = sim_yn_map %>% mutate(type = ifelse(type == "HT", 1, 0)) 

In [ ]:
fn_draws = paste0(here(), '/inputs/sim_sub_yn_logit_draws.csv')

if(file.exists(fn_draws)){
 sim_yn_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(sim_yn_map$subnum)
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  sim_yn_logit_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = sim_yn_map %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),# Coding of type: doesn't matter if it's a factor vs 0/1
             norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
      select(pred_yesChosen, norm_valDiff, norm_day, type)
    
    cur_m <- brm(formula = pred_yesChosen ~ norm_valDiff * norm_day * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    sim_yn_logit_draws = rbind(sim_yn_logit_draws, cur_draws)
  }
  names(sim_yn_logit_draws) = gsub(":", ".", names(sim_yn_logit_draws))
  write.csv(sim_yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
fn_draws = paste0(here(), '/inputs/yn_logit_norm_draws.csv')
if(!exists("yn_logit_draws")){
  yn_logit_draws = read.csv(fn_draws)
}

yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(b_Intercept, b_norm_valDiff, b_norm_day, b_type1, b_norm_valDiff.norm_day, b_norm_valDiff.type1, b_norm_day.type1, b_norm_valDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.norm_day" = "Value Diff x Day",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_valDiff.norm_day.type1" = "Value Diff x Day x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Day", "Value Diff x Type", "Day x Type", "Value Diff x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N (true)") 

sim_yn_par_summary = as_tibble(sim_yn_logit_draws) %>%
  select(b_Intercept, b_norm_valDiff, b_norm_day, b_type1, b_norm_valDiff.norm_day, b_norm_valDiff.type1, b_norm_day.type1, b_norm_valDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.norm_day" = "Value Diff x Day",
                    "b_norm_valDiff.type1" = "Value Diff x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_valDiff.norm_day.type1" = "Value Diff x Day x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Day", "Value Diff x Type", "Day x Type", "Value Diff x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N (sim)") 


In [ ]:
p = yn_par_summary %>%
  rbind(sim_yn_par_summary) %>%
  ggplot(aes(m, subnum, linetype = task, shape = task)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75), color = "blue")+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75), color = "blue")+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes/Left ~ Value Diff x Day x Type", shape = "", linetype = "") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  lemon::scale_x_symmetric(mid=0)+
  scale_linetype_manual(values = c("dashed", "solid"))+
  scale_shape_manual(values = c(5, 19))


# ggsave(file=paste0(fig_out_path, 'simsubyn_logit_ParEsts.jpg'), p, height = 5, width = 9, units="in")
p

In [ ]:
rm(yn_logit_draws, yn_par_summary)

- Is the value difference effect smallest on day 1 in predicted data?

In [ ]:
fn_draws = paste0(here(), '/inputs/sim_sub_yn_logit_norm_bydays_draws.csv')

if(file.exists(fn_draws)){
  sim_yn_logit_draws = read.csv(fn_draws)
} else{
  subnums = unique(sim_yn_map$subnum)
  days = list("day 1" = c(1), "day 2" = c(2), "day 3" = c(3), "week 2" = c(4, 5, 6, 7, 8), "week 3" = c(9, 10, 11))
  
  normMax = 1
  normMinDay = 0
  normMinVal = -1
  
  sim_yn_logit_draws = tibble()
  
  for(cur_sub in subnums){
    for (j in 1:length(days)){
      cur_days = days[[j]]
     
       cur_dat = sim_yn_map %>% 
        ungroup() %>%
        filter(subnum == cur_sub) %>%
        mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
               norm_day = (normMax - normMinDay) / (max(day) - min(day)) * (day - max(day)) + (normMax)) %>%
        group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
        mutate(norm_valDiff = (normMax - normMinVal) / (max(val_diff) - min(val_diff)) * (val_diff - max(val_diff)) + (normMax)) %>%
        filter(day %in% cur_days) %>%
        ungroup() %>%
        select(pred_yesChosen, norm_valDiff, norm_day, type)
      
      cur_m <- brm(formula = pred_yesChosen ~ norm_valDiff * type,
                   data= cur_dat,
                   family = bernoulli(link = "logit"),
                   warmup = 500,
                   iter = 2500,
                   chains = 4,
                   init = "0",
                   cores = 4,
                   seed = 389253)
      
      cur_draws = as_draws_df(cur_m)
      cur_draws$subnum = cur_sub
      cur_draws$day = names(days)[j]
      
      sim_yn_logit_draws = rbind(sim_yn_logit_draws, cur_draws)
    }
  
  }
  names(sim_yn_logit_draws) = gsub(":", ".", names(sim_yn_logit_draws))
  write.csv(sim_yn_logit_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
if(!exists("yn_logit_draws")){
  fn_draws = paste0(here(), '/inputs/yn_logit_norm_bydays_draws.csv')
  yn_logit_draws = read.csv(fn_draws)
}

yn_par_summary = as_tibble(yn_logit_draws) %>%
  select(subnum, day, b_Intercept, b_norm_valDiff, b_type1, b_norm_valDiff.type1) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.type1" = "Value Diff x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Type"))) %>%
  group_by(subnum, day, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "Y/N (true)")

sim_yn_par_summary = as_tibble(sim_yn_logit_draws) %>%
  select(subnum, day, b_Intercept, b_norm_valDiff, b_type1, b_norm_valDiff.type1) %>%
  mutate(day = factor(day, levels = c("day 1", "day 2", "day 3", "week 2", "week 3"))) %>%
  gather(key, value, -subnum, -day) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_valDiff"  = "Value Diff [-1, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_valDiff.type1" = "Value Diff x Type"),
         key = factor(key, levels = c("Intercept", "Value Diff [-1, 1]", "Type (RE = 0, HT = 1)", "Value Diff x Type"))) %>%
  group_by(subnum, day, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(task = "Y/N (sim)")

In [ ]:
p = yn_par_summary %>%
  rbind(sim_yn_par_summary) %>%
  ggplot(aes(day, m, shape = task, linetype = task,))+
  geom_point(position = position_dodge(width=.75), color="blue")+
  geom_errorbar(aes(ymin = l95, ymax = h95), width=.2, position = position_dodge(width=.75), color="blue")+
  facet_grid(key~subnum, scales="free")+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor = element_blank(),
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.position = "bottom",
        legend.box.margin=margin(t = -10))+
  labs(x="", y="Parameter Estimate", title="95% HDI for posterior slopes - Yes/Left ~ Value Diff x Type", shape = "", linetype = "")+
  scale_linetype_manual(values = c("dashed", "solid"))+
  scale_shape_manual(values = c(5, 19))+
  scale_alpha_manual(values = c(1, 1))

# ggsave(file=paste0(fig_out_path, 'simyn_logit_valdiff_slope.jpg'), p, height = 7, width = 10, units="in")
p

In [ ]:
rm(p, sim_yn_logit_draws, sim_yn_par_summary, yn_ddm_summary, yn_logit_draws, yn_par_summary)
rm(conds, days, fn_draws, stim_types, subs, types)

### Value difference on RT

RT: 
- Is the RT jump from day 1 to day 2 largest

In [ ]:
sim_group_summary = sim_yn_map %>%
  mutate(type_chr = ifelse(type == 1, "HT", "RE"),
         time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(pred_rt)),
            .groups="keep") %>%
  mutate(data = "Sim")

true_group_summary = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(time_bin, abs_val_diff_bin_str, type) %>%
  summarise(mean_y = mean(log(rt)),
            .groups="keep") %>%
  mutate(data = "True")

In [ ]:
p = ggplot() +
  geom_point(data = true_group_summary, aes(abs_val_diff_bin_str, mean_y, color = type, alpha = time_bin, size = time_bin))+
  geom_line(data = true_group_summary, aes(abs_val_diff_bin_str, mean_y, color = type, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task (True)", alpha="", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(-1,0.25,.25), limits = c(-1, .35))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'ynSimCompScale_group_MeanLogRTOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
p = ggplot() +
  geom_point(data = sim_group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, alpha = time_bin, size = time_bin))+
  geom_line(data = sim_group_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_wrap(~type_chr)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title="Y/N Task (Sim)", color="", shape="", alpha = "", size = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(-1,0.25,.25), limits = c(-1, .35))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))

# ggsave(file=paste0(fig_out_path, 'simsubyn_group_MeanLogRTOverValDiffAcrossTime.jpg'), p, height = 4, width = 8, units="in")

p

In [ ]:
sim_sub_summary = sim_yn_map %>%
  mutate(type_chr = ifelse(type == 1, "HT", "RE"),
         time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, time_bin, abs_val_diff_bin_str, type_chr) %>%
  summarise(mean_y = mean(log(pred_rt)),
            .groups="keep") %>%
  mutate(data = "Sim")


ggplot() +
  geom_point(data = sim_sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, size = time_bin, alpha = time_bin))+
  geom_line(data = sim_sub_summary, aes(abs_val_diff_bin_str, mean_y, color = type_chr, group = time_bin, alpha = time_bin)) +
  facet_grid(type_chr~subnum)+
  labs(x = "|Value Stim - Value Reference|", y = "Mean Log RT", title = "Y/N Task (Sim)", color="", size = "", alpha = "")+
  theme(panel.grid.major.x  = element_blank(),
        panel.grid.minor.x  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5), size = 12),
        axis.text.y = element_text(size = 12),
        strip.text = element_text(size = 12),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Dark2")+
  scale_y_continuous(breaks = seq(-1,0.25,.25))+
  guides(linetype = "none", color = "none")+
  scale_alpha_manual(values = c(1, .6, .5, .3, .2))+
  scale_size_manual(values = c(1.5, 2 , 2.5, 3, 3.5))


- Slope of absolute value difference on RT is negative?
- Slope of absolute value difference on RT smaller for RE than HT?

In [ ]:
fn_draws = paste0(here(), '/inputs/sim_yn_rt_norm_draws.csv')

if(file.exists(fn_draws)){
  sim_yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(sim_yn_map$subnum)
  normMax = 1
  normMin = 0
  
  sim_yn_rt_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = sim_yn_map %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type), # Coding of type: doesn't matter if it's a factor vs 0/1 integer
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             log_pred_rt = log(pred_rt)) %>%
      group_by(day) %>% # Grouping by day to normalize valdiff matters. This makes sure every day ranges [-1,1]
      mutate(norm_absValDiff = (normMax - normMin) / (max(abs_val_diff) - min(abs_val_diff)) * (abs_val_diff - max(abs_val_diff)) + (normMax)) %>%
      ungroup() %>%
      select(log_pred_rt, norm_absValDiff, norm_day, type)
    
    cur_m <- brm(formula = log_pred_rt ~ norm_absValDiff * norm_day * type,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    sim_yn_rt_draws = rbind(sim_yn_rt_draws, cur_draws)
  }
  names(sim_yn_rt_draws) = gsub(":", ".", names(sim_yn_rt_draws))
  write.csv(sim_yn_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
fn_draws = paste0(here(), '/inputs/yn_rt_norm_draws.csv')
if(!exists("yn_rt_draws")){
  yn_rt_draws = read.csv(fn_draws)
}

yn_par_summary = as_tibble(yn_rt_draws) %>%
  select(b_Intercept, b_norm_absValDiff, b_norm_day, b_type1, b_norm_absValDiff.norm_day, b_norm_absValDiff.type1, b_norm_day.type1, b_norm_absValDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_absValDiff.norm_day" = "|Value Diff| x Day",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_absValDiff.norm_day.type1" = "|Value Diff| x Day x Type"),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "|Value Diff| x Day", "|Value Diff| x Type", "Day x Type", "|Value Diff| x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N (true)") 

sim_yn_par_summary = as_tibble(sim_yn_rt_draws) %>%
  select(b_Intercept, b_norm_absValDiff, b_norm_day, b_type1, b_norm_absValDiff.norm_day, b_norm_absValDiff.type1, b_norm_day.type1, b_norm_absValDiff.norm_day.type1, subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key=recode(key, 
                    "b_Intercept" = "Intercept", 
                    "b_norm_absValDiff"  = "|Value Diff| [0, 1]",
                    "b_norm_day" = "Day [0, 1]",
                    "b_type1" = "Type (RE = 0, HT = 1)",
                    "b_norm_absValDiff.norm_day" = "|Value Diff| x Day",
                    "b_norm_absValDiff.type1" = "|Value Diff| x Type",
                    "b_norm_day.type1" = "Day x Type",
                    "b_norm_absValDiff.norm_day.type1" = "|Value Diff| x Day x Type"),
         key = factor(key, levels = c("Intercept", "|Value Diff| [0, 1]", "Day [0, 1]", "Type (RE = 0, HT = 1)", "|Value Diff| x Day", "|Value Diff| x Type", "Day x Type", "|Value Diff| x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N (sim)")


In [ ]:
plim = max(abs(sim_yn_par_summary$l95), abs(yn_par_summary$l95) ,sim_yn_par_summary$h95, yn_par_summary$h95)

p = yn_par_summary %>%
  rbind(sim_yn_par_summary) %>%
  ggplot(aes(m, subnum, linetype = task, shape = task, alpha = task)) +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75), color = "blue")+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75), color = "blue")+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ |Value Diff| x Day x Type", linetype = "", shape = "") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(limits = c(-plim, plim))+
  scale_linetype_manual(values = c("dashed", "solid"))+
  scale_shape_manual(values = c(5, 19))+
  scale_alpha_manual(values = c(1, 1))+
  guides(alpha = "none")

# ggsave(file=paste0(fig_out_path, 'simyn_rt_ParEsts.jpg'), p, height = 5, width = 9, units="in")
p

In [ ]:
rm(yn_rt_draws, yn_par_summary)

## Parameter changes across days

In [ ]:
stim_types = c("HT", "RE")
subnums = c(601, 609, 611, 619, 621, 629)

in_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/jags_out'

yn_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(subnums)){
    
    cur_stim_type = stim_types[i]
    cur_sub = subnums[j]
    # summary_YN_HDDM_FIT_RE_sub-611.csv
    
    fn = file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_sub-', cur_sub,'.csv'))
    
    if(file.exists(fn)){
      cur_summary = read.csv(fn)
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               subnum = cur_sub)
      
      yn_ddm_summary = rbind(yn_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}

In [ ]:
yn_ddm_summary = yn_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, subnum, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_group = gsub("p\\[", "", par_group),
         par_group = gsub("\\]", "", par_group)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  rename(day = par_group) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name)))) %>%
  mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt")),
         day = as.numeric(day))

In [ ]:
p = yn_ddm_summary %>%
  ggplot(aes(day, Mean, color=stim_type))+
  geom_point(position = position_dodge(width=.75))+
  geom_errorbar(aes(ymin = Lower95, ymax = Upper95), position = position_dodge(width=.75), width=.5)+
  facet_grid(par_name ~ subnum, scale = "free_y")+
  scale_color_brewer(palette = "Dark2")+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10))+
  labs(x="Day", y = "Parameter Estimate", color="", title = "YN HDDM Estimates (95% HDI) - Hierarchy over days")+
  scale_x_continuous(breaks = seq(1,11,1),
                     labels = c("1", "", "3", "", "5", "", "7", "", "9", "", "11"))

# ggsave(file=paste0(fig_out_path, 'yn_subj_hddm_parests_sub-hierarchy.jpg'), p, height = 6, width=9, units="in")
p